### Some additional data cleaning
I realized that my data was incomplete so I set out in this notebook to reconstruct the missing data.

In [ ]:
import pandas as pd
#read in the csv files and store them in a DataFrame
df = pd.read_csv('G:\\StanfordCarsDatasetCSV-main\\cardatasettrain.csv', index_col = 0)
test_df = pd.read_csv('G:\\StanfordCarsDatasetCSV-main\\cardatasettest.csv', index_col = 0) 
car_names = pd.read_csv('G:\\StanfordCarsDatasetCSV-main\\class_names.csv')
# Print the DataFrame
print(df)
print(test_df)

In [ ]:
car_names.head()

In [ ]:
#add the car names to the DataFrame using the class column and the index of the car_names DataFrame
df['Car Name'] = df['Class'].apply(lambda x: car_names.loc[x-1, 'class_description'])
print(df.head())

At this point I thought I might be missing something so I went back to the original data source to double check.

In [ ]:
#pulled from the other preprocessing notebook
data = scipy.io.loadmat('G:\\\\capstone files\\\\data\\\\archive (1)\\\\cars_annos.mat')
#make dataframes for each column to make it easier to work with
relative_im_path = pd.DataFrame(data['annotations']['relative_im_path']).transpose()
bbox_x1 = pd.DataFrame(data['annotations']['bbox_x1']).transpose()
bbox_x2 = pd.DataFrame(data['annotations']['bbox_x2']).transpose()
bbox_y1 = pd.DataFrame(data['annotations']['bbox_y1']).transpose()
bbox_y2 = pd.DataFrame(data['annotations']['bbox_y2']).transpose()
class_id = pd.DataFrame(data['annotations']['class']).transpose()
test = pd.DataFrame(data['annotations']['test']).transpose()
#concatenate the dataframes into one
annotations = pd.concat([relative_im_path, bbox_x1, bbox_y1, bbox_x2, bbox_y2, class_id, test], axis=1)
annotations.columns = ['relative_im_path', 'x1', 'y1', 'x2', 'y2', 'class_id', 'test']

#values are in the same order, confirmed through the matlab file using octave
#cast all columns to int except for relative_im_path
annotations[annotations.columns.difference(['relative_im_path'])] = annotations[annotations.columns.difference(['relative_im_path'])].astype(int)
annotations.head()

In [ ]:
#checking for duplicates
duplicates = annotations.duplicated(subset=annotations.columns.difference(['relative_im_path']), keep=False)
duplicated_rows = annotations[duplicates]
print(duplicated_rows)


one potentially mismatched row will not ruin the whole dataset, the image path is incorrect so I cannot check to see which one is correct.

In [ ]:
#find all rows where the x1, y1, x2, and y2 values are the same
test_matching_rows = test_df.merge(annotations, on=['x1', 'y1', 'x2', 'y2'])
#merge the two DataFrames on the x1, y1, x2, and y2 columns
#add the car names to the DataFrame using the class column and the index of the car_names DataFrame
test_matching_rows['Car Name'] = test_matching_rows['class_id'].apply(lambda x: car_names.loc[x-1, 'class_description'])

test_matching_rows.head()

I realized that I did not have complete data for each train and test set. I found a .csv version of the matlab file online but it did not have anything except the bounding box coordinates. So I used the matlab file to reconstruct the rest of the data by matching the bounding box coordinates.

In [ ]:
train_matching_rows = df.merge(annotations, on=['x1', 'y1', 'x2', 'y2'])
train_matching_rows.head()


Just checking that I'm not missing values in the training set.

In [ ]:
print(train_matching_rows['test'].value_counts())
print(test_matching_rows['test'].value_counts())


Checking to see if I have any misclassified items. I decided to leave them because
later on I realized that the resizing function that I made later doesn't work without them.

In [ ]:
#dropping redundant columns
train_matching_rows.drop(['relative_im_path', 'Class', 'test'], axis=1, inplace=True)
test_matching_rows.drop(['relative_im_path', 'test'], axis=1, inplace=True)

In [ ]:
#rename columns
train_matching_rows = train_matching_rows[['image', 'x1', 'y1', 'x2', 'y2', 'Car Name', 'class_id']]
test_matching_rows = test_matching_rows[['image', 'x1', 'y1', 'x2', 'y2', 'Car Name', 'class_id']]
print(train_matching_rows.head())
print(test_matching_rows.head())

In [ ]:
from PIL import Image
import glob
# FILEPATH: Untitled-1.ipynb
train_image_paths = glob.glob('G:/capstone files/data/archive (1)/cars_train/cars_train/*.jpg')
test_image_paths = glob.glob('G:/capstone files/data/archive (1)/cars_test/cars_test/*.jpg')

# Get image size for train images
train_image_sizes = []
for image_path in train_image_paths:
    image = Image.open(image_path)
    width, height = image.size
    wsf = 224 / width
    hsf = 224 / height
    train_matching_rows.loc[train_matching_rows['image'] == image_path.split('\\')[-1], ['x1', 'x2']] *= wsf
    train_matching_rows.loc[train_matching_rows['image'] == image_path.split('\\')[-1], ['y1', 'y2']] *= hsf

# Get image size for test images
test_image_sizes = []
for image_path in test_image_paths:
    image = Image.open(image_path)
    width, height = image.size
    wsf = 224 / width
    hsf = 224 / height
    test_matching_rows.loc[test_matching_rows['image'] == image_path.split('\\')[-1], ['x1', 'x2']] *= wsf
    test_matching_rows.loc[test_matching_rows['image'] == image_path.split('\\')[-1], ['y1', 'y2']] *= hsf





This code gets the sizes of the images, and then uses that to rescale the bounding boxes so that they fit on the resized images.

In [ ]:
import gc
gc.collect()
# this is here to reduce RAM usage

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Set the directory path
directory = 'G:/capstone files/data/cars train resized'

# Get the file names in the directory
file_names = os.listdir(directory)

# Show the first 5 images
for i in range(5):
    image_path = os.path.join(directory, file_names[i])
    image = plt.imread(image_path)
    
    # Create a figure and axes
    fig, ax = plt.subplots()
    
    # Display the image
    ax.imshow(image)
    
    # Get the bounding box coordinates
    x1 = train_matching_rows.loc[i, 'x1']
    y1 = train_matching_rows.loc[i, 'y1']
    x2 = train_matching_rows.loc[i, 'x2']
    y2 = train_matching_rows.loc[i, 'y2']
    
    # Create a rectangle patch
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
    
    # Add the rectangle to the plot
    ax.add_patch(rect)
    
    # Turn off the axis labels
    ax.axis('off')
    
    # Show the plot
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Set the directory path
directory = 'G:/capstone files/data/cars test resized'

# Get the file names in the directory
file_names = os.listdir(directory)

# Show the first 5 images
for i in range(5):
    image_path = os.path.join(directory, file_names[i])
    image = plt.imread(image_path)
    
    # Create a figure and axes
    fig, ax = plt.subplots()
    
    # Display the image
    ax.imshow(image)
    
    # Get the bounding box coordinates
    x1 = test_matching_rows.loc[i, 'x1']
    y1 = test_matching_rows.loc[i, 'y1']
    x2 = test_matching_rows.loc[i, 'x2']
    y2 = test_matching_rows.loc[i, 'y2']
    
    # Create a rectangle patch
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
    
    # Add the rectangle to the plot
    ax.add_patch(rect)
    
    # Turn off the axis labels
    ax.axis('off')
    
    # Show the plot
    plt.show()

This code just checks to see if the boxes fit.

In [ ]:
train_matching_rows.to_csv('G:\capstone files\data\\train_final.csv', index=False)
test_matching_rows.to_csv('G:\capstone files\data\\test_final.csv', index=False)
